In [50]:
# please ignore warning messages during the installation
!pip install --disable-pip-version-check -q sagemaker==2.35.0
!conda install -q -y pytorch==1.6.0 -c pytorch
!pip install --disable-pip-version-check -q transformers==3.5.1
!pip install -q protobuf==3.20.*

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: ...working... failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package pytorch-1.6.0-py3.6_cpu_0 requires python >=3.6,<3.7.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ pin-1 is installable and it requires
│  └─ python 3.11.* , which can be installed;
└─ pytorch 1.6.0  is not installable because there are no viable options
   ├─ pytorch 1.6.0 would require
   │  └─ python >=3.6,<3.7.0a0 , which conflicts with any installable versions previously reported;
   ├─ pytorch 1.6.0 would require
   │  └─ python >=3.7,<3.8.0a0 , which conflicts with any installable versions previously reported;
   ├─ pytorch 1.6.0 would require
   │  └─ python >=3.8,<3.9.0a0 , which conflicts with any installable versions previously reported;
   └─ pytorch 1.6.0 would require
      └─ python >=3.9,<3.10.0a0 , which conflicts w

In [51]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [52]:
import boto3
import sagemaker
import pandas as pd
import botocore

In [53]:
# ========================== low-level service client of the boto3 session ==========================
config = botocore.config.Config(user_agent_extra='bedissj-1699438736259')


sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

cw = boto3.client(service_name='cloudwatch', 
                  config=config)

autoscale = boto3.client(service_name="application-autoscaling", 
                         config=config)

## 1. Create an endpoint with multiple variants 

In [54]:
model_a_s3_uri = "s3://{}/y8a6c91lumhh-ModelTra-GTH766MIe3-002-1a2991b7/output/model.tar.gz".format(bucket)
model_b_s3_uri = "s3://{}/y8a6c91lumhh-ModelTra-GTH766MIe3-010-b1656de1/output/model.tar.gz".format(bucket)

### 1.1 Construct Docker Image URI

In [55]:
FRAMEWORK_VERSION = '0.20.0'
deploy_instance_type = "ml.m5.large"

In [56]:
churn_inference_image_uri = sagemaker.image_uris.retrieve(
    framework='sklearn',
    version=FRAMEWORK_VERSION,
    instance_type=deploy_instance_type,
    image_scope="inference",
    region=region
)

print(churn_inference_image_uri)

659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3


In [57]:
import time

timestamp = int(time.time())

model_name_a = "churn-prediction-mon1-model-a-{}".format(timestamp)
model_name_b = "churn-prediction-mon1-model-b-{}".format(timestamp)


Check if models already exist.

In [58]:
def check_model_existence(model_name):
    for model in sm.list_models()['Models']:
        if model_name == model['ModelName']:
            return True
    return False

### 1.2 Create models

In [74]:
from sagemaker.sklearn.model import SKLearnModel, FrameworkModel
from pprint import pprint


if not check_model_existence(model_name=model_name_a):
    model_a = SKLearnModel(
        name=model_name_a,
        model_data=model_a_s3_uri,
        image_uri=churn_inference_image_uri,
        entry_point='./src/inference.py',
        sagemaker_session=sess,
        role=role
    )
    pprint(model_a)

else:
    print("Model {} already exists".format(model_name_a))

In [75]:
if not check_model_existence(model_name=model_name_b):
    # model_b = sm.create_model(
    #     ModelName=model_name_b,
    #     ExecutionRoleArn=role,
    #     PrimaryContainer={
    #         "ModelDataUrl": model_b_s3_uri,
    #         "Image":churn_inference_image_uri,
    #     }
    # )

    model_b = FrameworkModel(
        name=model_name_b,
        model_data=model_b_s3_uri,
        image_uri=churn_inference_image_uri,
        entry_point='./src/inference.py',
        sagemaker_session=sess,
        role=role
    )
    pprint(model_b)

else:
    print("Model {} already exists".format(model_name_b))

### 1.3 Create production variants

In [61]:
from sagemaker.session import production_variant

variantA = production_variant(
    model_name=model_name_a,
    instance_type=deploy_instance_type,
    initial_instance_count=1,
    initial_weight=50,
    variant_name='VariantA'
)
print(variantA)

{'ModelName': 'churn-prediction-mon1-model-a-1728467686', 'InstanceType': 'ml.m5.large', 'InitialInstanceCount': 1, 'VariantName': 'VariantA', 'InitialVariantWeight': 50}


In [62]:
variantB = production_variant(
    model_name=model_name_b,
    instance_type=deploy_instance_type,
    initial_instance_count=1,
    initial_weight=50,
    variant_name='VariantB'
)
print(variantA)

{'ModelName': 'churn-prediction-mon1-model-a-1728467686', 'InstanceType': 'ml.m5.large', 'InitialInstanceCount': 1, 'VariantName': 'VariantA', 'InitialVariantWeight': 50}


### 1.4 Configure and create the endpoint

Check endpoint configuration existence.

In [63]:
def check_endpoint_config_existence(endpoint_config_name):
    for endpoint_config in sm.list_endpoint_configs()['EndpointConfigs']:
        if endpoint_config_name == endpoint_config['EndpointConfigName']:
            return True
    return False


def check_endpoint_existence(endpoint_name):
    for endpoint in sm.list_endpoints()['Endpoints']:
        if endpoint_name == endpoint['EndpointName']:
            return True
    return False

Create endpoint configuration for  A/B testing.

In [64]:
endpoint_config_name = "churn-prediction-ab-epc-{}".format(timestamp)

if not check_endpoint_config_existence(endpoint_config_name):
    endpoint_config = sm.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[variantA, variantB]
    )
    pprint(endpoint_config)
else: 
     print("Endpoint configuration {} already exists".format(endpoint_config_name))

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Could not find model "churn-prediction-mon1-model-a-1728467686".

In [80]:
sm.list_models()

{'Models': [],
 'ResponseMetadata': {'RequestId': 'df7aaca5-0d29-4b35-9fe3-8f110aa2b029',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'df7aaca5-0d29-4b35-9fe3-8f110aa2b029',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '13',
   'date': 'Wed, 09 Oct 2024 10:13:25 GMT'},
  'RetryAttempts': 0}}

In [79]:
model_a = Model(
        name="test",
        model_data=model_a_s3_uri,
        image_uri=churn_inference_image_uri,
        sagemaker_session=sess,
        role=role
    )

In [87]:
sm.create_model?

Signature: sm.create_model(*args, **kwargs)
Docstring:
Creates a model in SageMaker. In the request, you name the model and describe a primary container. For the primary container, you specify the Docker image that contains inference code, artifacts (from prior training), and a custom environment map that the inference code uses when you deploy the model for predictions.

 

Use this API to create a model if you want to use SageMaker hosting services or run a batch transform job.

 

To host your model, you create an endpoint configuration with the ``CreateEndpointConfig`` API, and then create an endpoint with the ``CreateEndpoint`` API. SageMaker then deploys all of the containers that you defined for the model in the hosting environment.

 

To run a batch transform using your model, you start a job with the ``CreateTransformJob`` API. SageMaker uses your model and your dataset to get inferences which are then saved to a specified S3 location.

 

In the request, you also provide an 